In [34]:
import os
from pathlib import Path
import torch
import re
import random
import tqdm
from torch.utils.data import Dataset, DataLoader
import itertools
import math
import torch.nn.functional as F
import numpy as np
from torch.optim import Adam

In [391]:
class BERTDataset_For_League(Dataset):
    def __init__(self, data_pair, seq_len=13):
        
        self.seq_len = seq_len
        self.corpus_lines = len(data_pair)
        self.lines = data_pair

    def __len__(self):
        return self.corpus_lines

    def __getitem__(self, item):

        # Step 1: get random sentence pair, either negative or positive (saved as is_next_label)
        t1, t2, winner_label = self.get_sent(item)

        # Step 2: replace random words in sentence with mask / random words
        t1_random, t1_label = self.masking_word(t1)
        t2_random, t2_label = self.masking_word(t2)

        # Step 3: Adding CLS(1) and SEP(2) tokens to the start and end of sentences (Don't need padding since all input have the same size)
        t1 = [1] + t1_random + [2]
        t2 = t2_random + [2]
        
        # Step 4: combine sentence 1 and 2 as one input
        segment_label = ([1 for _ in range(len(t1))] + [2 for _ in range(len(t2))])[:self.seq_len]
        bert_input = (t1 + t2)[:self.seq_len]
        bert_label = ([1] + t1_label + [2] + t2_label + [2])[:self.seq_len]
        
        output = {"bert_input": bert_input,
                  "bert_label": bert_label,
                  "segment_label": segment_label,
                  "winner_label": winner_label}

        return {key: torch.tensor(value) for key, value in output.items()}

    def masking_word(self, sentence):
        tokens = sentence
        output_label = []
        output = []

        # 15% of the tokens would be replaced
        for i, token in enumerate(tokens):
            prob = random.random()
            token_id = [int(token)]
            
            if prob < 0.15:
                prob /= 0.15

                # 80% chance change token to mask(3) token
                if prob < 0.8:
                    for i in range(len(token_id)):
                        output.append(3)

                # 20% chance change token to current token
                else:
                    output.append(token_id)

                output_label.append(token_id)

            else:
                output.append(token_id)
                for i in range(len(token_id)):
                    output_label.append(0)

        # flattening
        output = list(itertools.chain(*[[x] if not isinstance(x, list) else x for x in output]))
        output_label = list(itertools.chain(*[[x] if not isinstance(x, list) else x for x in output_label]))
        assert len(output) == len(output_label)
        return output, output_label

    def get_sent(self, index):
        '''return random sentence pair'''
        t1, t2, label = self.get_corpus_line(index)
        return t1, t2, label

    def get_corpus_line(self, item):
        '''return sentence pair'''
        return self.lines[item][0], self.lines[item][1], self.lines[item][2]

    # def get_random_line(self):
    #     '''return random single sentence'''
    #     return self.lines[random.randrange(len(self.lines))][1]

In [392]:
class PositionalEmbedding(torch.nn.Module):
    
    def __init__(self, d_model, max_len=128):
        super().__init__()

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model).float()
        pe.require_grad = False

        for pos in range(max_len):   
            # for each dimension of the each position
            for i in range(0, d_model, 2):   
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))

        # include the batch size
        self.pe = pe.unsqueeze(0)   
        # self.register_buffer('pe', pe)

    def forward(self, x):
        return self.pe

class BERTEmbedding(torch.nn.Module):
    """
    BERT Embedding which is consisted with under features
        1. TokenEmbedding : normal embedding matrix
        2. PositionalEmbedding : adding positional information using sin, cos
        2. SegmentEmbedding : adding sentence segment info, (sent_A:1, sent_B:2)
        sum of all these features are output of BERTEmbedding
    """

    def __init__(self, vocab_size, embed_size, seq_len=13, dropout=0.1):
        """
        :param vocab_size: total vocab size
        :param embed_size: embedding size of token embedding
        :param dropout: dropout rate
        """

        super().__init__()
        self.embed_size = embed_size
        # (m, seq_len) --> (m, seq_len, embed_size)
        # padding_idx is not updated during training, remains as fixed pad (0)
        self.token = torch.nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.segment = torch.nn.Embedding(3, embed_size, padding_idx=0)
        self.position = PositionalEmbedding(d_model=embed_size, max_len=seq_len)
        self.dropout = torch.nn.Dropout(p=dropout)
       
    def forward(self, sequence, segment_label):
        x = self.token(sequence) + self.position(sequence) + self.segment(segment_label)
        return self.dropout(x)

In [393]:
from torch.nn import MultiheadAttention


class FeedForward(torch.nn.Module):
    "Implements FFN equation."

    def __init__(self, d_model, middle_dim=2048, dropout=0.1):
        super(FeedForward, self).__init__()
        
        self.fc1 = torch.nn.Linear(d_model, middle_dim)
        self.fc2 = torch.nn.Linear(middle_dim, d_model)
        self.dropout = torch.nn.Dropout(dropout)
        self.activation = torch.nn.GELU()

    def forward(self, x):
        out = self.activation(self.fc1(x))
        out = self.fc2(self.dropout(out))
        return out

class EncoderLayer(torch.nn.Module):
    def __init__(
        self, 
        d_model=768,
        heads=12, 
        feed_forward_hidden=768 * 4, 
        dropout=0.1
        ):
        super(EncoderLayer, self).__init__()
        self.layernorm = torch.nn.LayerNorm(d_model)
        self.self_multihead = MultiheadAttention(d_model, heads, batch_first=True)
        self.feed_forward = FeedForward(d_model, middle_dim= d_model *4)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, embeddings, mask = None):
        # embeddings: (batch_size, max_len, d_model)
        # encoder mask: (batch_size, 1, 1, max_len)
        # result: (batch_size, max_len, d_model)
        attention_res = self.self_multihead(embeddings, embeddings, embeddings, mask)
       

        interacted = self.dropout(attention_res[0])
        # residual layer
        interacted = self.layernorm(interacted + embeddings)
        # bottleneck
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        encoded = self.layernorm(feed_forward_out + interacted)
        return encoded

In [394]:
class BERT(torch.nn.Module):
    def __init__(self, vocab_size, d_model = 768, n_layers = 12, heads = 12, dropout = 0.1):
        """
        :param vocab_size: vocab_size of total words
        :param hidden: BERT model hidden size (d_model * n)
        :param n_layers: numbers of Transformer blocks (Encoder layers)
        :param attn_heads: number of attention heads
        dropout: dropout rate
        """

        super().__init__()
        self.d_model = d_model
        self.n_layers = n_layers
        self.heads = heads
        
        self.feed_forward_hidden = d_model * 4

        self.embedding = BERTEmbedding(vocab_size = vocab_size, embed_size = d_model, seq_len = 13)

        self.encoder_blocks = torch.nn.ModuleList([
            EncoderLayer(d_model, heads, d_model * 4, dropout) for _ in range(n_layers)
        ])

    def forward(self, x, segment_info):
        # mask = (x > 0).unsqueeze(1).repeat(1, x.size(1), 1).unsqueeze(1)
        mask = (x == 0)
        # print(type(x==0))

        x = self.embedding(x, segment_info)
        for encoder in self.encoder_blocks:
            x = encoder.forward(x, mask)
        return x
    
class WinnerPrediction(torch.nn.Module):
    def __init__(self, hidden):
        super().__init__()
        self.linear = torch.nn.Linear(hidden, 2)
        self.softmax = torch.nn.LogSoftmax(dim = -1)
    
    def forward(self, x):
        return self.softmax(self.linear(x[:, 0]))

class MaskedLanguageModel(torch.nn.Module):
    def __init__(self, hidden, vocab_size):
        super().__init__()
        self.linear = torch.nn.Linear(hidden, vocab_size)
        self.softmax = torch.nn.LogSoftmax(dim = - 1)
    
    def forward(self, x):
        return self.softmax(self.linear(x))
    
class BERTLM(torch.nn.Module):
    def __init__(self, bert: BERT, vocab_size):
        super().__init__()
        self.bert = bert
        self.predict_winner = WinnerPrediction(self.bert.d_model)
        self.mask_lm = MaskedLanguageModel(self.bert.d_model, vocab_size)
    
    def forward(self, x, segment_label):
        x = self.bert(x, segment_label)
        return self.predict_winner(x), self.mask_lm(x)


In [395]:
class ScheduledOptim():
    def __init__(self, optimizer, d_model, n_warmup_steps):
        self.optimizer = optimizer
        self.n_warmup_steps = n_warmup_steps
        self.n_current_steps = 0
        self.init_lr = np.power(d_model, -0.5)

    def step_and_update_lr(self):
        self.update_learning_rate()
        self.optimizer.step()
    
    def zero_grad(self):
        self.optimizer.zero_grad()

    def get_lr_scale(self):
        return np.min([
            np.power(self.n_current_steps, -0.5),
            np.power(self.n_warmup_steps, -1.5) * self.n_current_steps
            ])

    def update_learning_rate(self):
        self.n_current_steps += 1
        lr = self.init_lr * self.get_lr_scale()

        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr

            

In [407]:
class BERTTrainer:
    def __init__(
        self, 
        model, 
        train_dataloader, 
        test_dataloader=None, 
        lr= 1e-4,
        weight_decay=0.01,
        betas=(0.9, 0.999),
        warmup_steps=10000,
        log_freq=10,
        device='cuda'
        ):

        self.device = device
        self.model = model
        self.train_data = train_dataloader
        self.test_data = test_dataloader

        # Setting the Adam optimizer with hyper-param
        self.optim = Adam(self.model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)
        self.optim_schedule = ScheduledOptim(
            self.optim, self.model.bert.d_model, n_warmup_steps=warmup_steps
            )

        # Using Negative Log Likelihood Loss function for predicting the masked_token
        self.criterion = torch.nn.NLLLoss(ignore_index=0)
        self.log_freq = log_freq
        print("Total Parameters:", sum([p.nelement() for p in self.model.parameters()]))
    
    def train(self, epoch):
        self.iteration(epoch, self.train_data)

    def test(self, epoch):
        self.iteration(epoch, self.test_data, train=False)

    def iteration(self, epoch, data_loader, train=True):
        
        avg_loss = 0.0
        total_correct = 0
        total_element = 0
        
        mode = "train" if train else "test"

        # progress bar
        data_iter = tqdm.tqdm(
            enumerate(data_loader),
            desc="EP_%s:%d" % (mode, epoch),
            total=len(data_loader),
            bar_format="{l_bar}{r_bar}"
        )

        for i, data in data_iter:

            # 0. batch_data will be sent into the device(GPU or cpu)
            data = {key: value.to(self.device) for key, value in data.items()}

            # 1. forward the next_sentence_prediction and masked_lm model
            next_sent_output, mask_lm_output = self.model.forward(data["bert_input"], data["segment_label"])

            # 2-1. NLL(negative log likelihood) loss of is_next classification result
            next_loss = self.criterion(next_sent_output, data["winner_label"])

            # 2-2. NLLLoss of predicting masked token word
            # transpose to (m, vocab_size, seq_len) vs (m, seq_len)
            # criterion(mask_lm_output.view(-1, mask_lm_output.size(-1)), data["bert_label"].view(-1))
            mask_loss = self.criterion(mask_lm_output.transpose(1, 2), data["bert_label"])

            # 2-3. Adding next_loss and mask_loss : 3.4 Pre-training Procedure
            loss = next_loss + mask_loss

            # 3. backward and optimization only in train
            if train:
                self.optim_schedule.zero_grad()
                loss.backward()
                self.optim_schedule.step_and_update_lr()

            # next sentence prediction accuracy
            correct = next_sent_output.argmax(dim=-1).eq(data["winner_label"]).sum().item()
            avg_loss += loss.item()
            total_correct += correct
            total_element += data["winner_label"].nelement()

            post_fix = {
                "epoch": epoch,
                "iter": i,
                "avg_loss": avg_loss / (i + 1),
                "avg_acc": total_correct / total_element * 100,
                "loss": loss.item()
            }

            if i % self.log_freq == 0:
                data_iter.write(str(post_fix))
        print(
            f"EP{epoch}, {mode}: \
            avg_loss={avg_loss / len(data_iter)}, \
            total_acc={total_correct * 100.0 / total_element}"
        ) 


In [408]:
# import pandas as pd
# import ast


# df = pd.read_csv("../Data/match_data_2.csv")
# df["teams"] = df["teams"].apply(lambda x : ast.literal_eval(x))


# champ_idx = {}
# with open("champ_idx.txt", 'r') as file:
#     for line in file:
#         pair = ast.literal_eval(line)
#         champ_idx.update({pair[0]: pair[1]})



# for i in range(len(df["teams"])):
#     for j in range(len(df["teams"][i])):
#         df["teams"][i][j] = champ_idx[df["teams"][i][j]]


# train_datas = [[df["teams"][i][:5], df["teams"][i][5:], df["winner"][i]] for i in range(len(df))]




In [409]:
# champ_idx

In [410]:
# with open("../Data/train_data.txt", 'w') as file:
#     for item in train_datas:
#         file.write(("%s\n" % item))

In [411]:
# temp = []
# for i, j, _ in train_datas:
#     temp = temp+i+ j

In [412]:
# temp_s  = sorted(list(set(temp)))

In [413]:
# champ_idx = {id: i for i, id in enumerate(temp_s)}

In [414]:
# with open("champ_idx.txt", 'w') as file:
#     for pair in champ_idx.items():
#         file.write(f"{pair} \n")

In [415]:
# train_datas

In [424]:
'''test run'''

MAX_LEN = 13

train_data = BERTDataset_For_League(
   train_datas, seq_len=MAX_LEN)

# print(train_data.)

train_loader = DataLoader(
   train_data, batch_size=32, shuffle=True, pin_memory=True)

bert_model = BERT(
  vocab_size=167,
  d_model= 64,
  n_layers=2,
  heads=8,
  dropout=0.1
)

bert_lm = BERTLM(bert_model, 167)
bert_trainer = BERTTrainer(bert_lm, train_loader, device='cpu')
epochs = 20

for epoch in range(epochs):
  bert_trainer.train(epoch)




Total Parameters: 121577


EP_train:0:   0%|| 0/938 [00:00<?, ?it/s]

EP_train:0:   1%|| 5/938 [00:00<00:22, 41.04it/s]

{'epoch': 0, 'iter': 0, 'avg_loss': 5.595807075500488, 'avg_acc': 40.625, 'loss': 5.595807075500488}


EP_train:0:   1%|| 14/938 [00:00<00:24, 37.04it/s]

...

{'epoch': 19, 'iter': 910, 'avg_loss': 1.46122594683152, 'avg_acc': 54.53485181119648, 'loss': 1.4913978576660156}


EP_train:19:  99%|| 927/938 [00:33<00:00, 26.77it/s]

{'epoch': 19, 'iter': 920, 'avg_loss': 1.4611185101810418, 'avg_acc': 54.52972312703584, 'loss': 1.4980571269989014}


EP_train:19: 100%|| 938/938 [00:33<00:00, 27.72it/s]

{'epoch': 19, 'iter': 930, 'avg_loss': 1.460711345636755, 'avg_acc': 54.56162728249194, 'loss': 1.5303101539611816}
EP19, train:             avg_loss=1.460497917905291,             total_acc=54.57485828609536


In [428]:
# train_data = BERTDataset_For_League(
#    train_datas, seq_len=MAX_LEN)

# # print(train_data.)

# train_loader = DataLoader(
#    train_data, batch_size=32, shuffle=True, pin_memory=True)

epochs = 101

for epoch in range(100, epochs):
  bert_trainer.train(epoch)

EP_train:100:   1%|| 9/938 [00:00<00:23, 39.47it/s]

{'epoch': 100, 'iter': 0, 'avg_loss': 1.5695569515228271, 'avg_acc': 56.25, 'loss': 1.5695569515228271}


EP_train:100:   2%|| 18/938 [00:00<00:24, 37.92it/s]

{'epoch': 100, 'iter': 10, 'avg_loss': 1.3758442835374312, 'avg_acc': 52.27272727272727, 'loss': 1.1518754959106445}


EP_train:100:   2%|| 22/938 [00:00<00:23, 38.38it/s]

...

{'epoch': 100, 'iter': 920, 'avg_loss': 1.4204289277653481, 'avg_acc': 54.54668838219327, 'loss': 1.2520614862442017}


EP_train:100: 100%|| 936/938 [00:34<00:00, 28.24it/s]

{'epoch': 100, 'iter': 930, 'avg_loss': 1.4210131492317684, 'avg_acc': 54.53813104189044, 'loss': 1.4338897466659546}


EP_train:100: 100%|| 938/938 [00:35<00:00, 26.79it/s]

EP100, train:             avg_loss=1.4216391075648733,             total_acc=54.57485828609536


In [427]:
torch.save(bert_lm.state_dict(), "bert_model")


In [421]:
# train_data = BERTDataset_For_League(
#    train_datas[:32], seq_len=MAX_LEN)

# # print(train_data.)

# train_loader = DataLoader(
#    train_data, batch_size=32, shuffle=True, pin_memory=True)



[[[142, 144, 33, 130, 121], [101, 120, 114, 71, 42], 0],
 [[136, 123, 130, 165, 137], [159, 100, 69, 161, 36], 1]]

In [406]:
18.69 + 3.99 + 7.99 + 19.53 + 8.89 + 15.99 -5 +32.97 + 2.54

105.59

In [145]:
13.99 + 5.99 + 9.49 + 12.79 + 19.24 + 16.99 - 4.5 + 29.99 +2.53

106.50999999999999